## MPI4Py example

In this example we use [IPCMagic](https://github.com/eth-cscs/ipcluster_magic/tree/master) to run an example from [MPI4Py's homepage](https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication) within a Jupyter notebook.

* From a shell in Piz Daint this can be run using this Slurm job script:
 
```
#!/bin/bash -l

#SBATCH --job-name=osubw
#SBATCH --time=00:05:00
#SBATCH --nodes=2
#SBATCH --ntasks-per-core=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=12
#SBATCH --partition=normal
#SBATCH --constraint=gpu
#SBATCH --account=<project>

# source python environment with cupy and mpi4py

srun python example.py
```

In [1]:
import os
import ipcmagic

In [2]:
%ipcluster start -n 2

100%|██████████| 2/2 [00:06<00:00,  3.20s/engine]


In [3]:
%%px
import socket

socket.gethostname()

Out[1:1]: 'nid07225'

Out[0:1]: 'nid07225'

In [4]:
# Disable IPyParallel's progress bar
%pxconfig --progress-after -1

In [5]:
%%px
from mpi4py import MPI
import numpy

In [6]:
%%px
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

In [7]:
%%px

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1)
elif rank == 1:
    data = numpy.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0)
    print(f' * array from rank {rank}: {data}')

[stdout:0]  * array from rank 1: [0 1 2 3 4 5 6 7 8 9]


In [8]:
%%px

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(10, dtype=numpy.float64)
    comm.Send(data, dest=1)
elif rank == 1:
    data = numpy.empty(10, dtype=numpy.float64)
    comm.Recv(data, source=0)
    print(f' * array from rank {rank}: {data}')

[stdout:0]  * array from rank 1: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [9]:
%ipcluster stop

IPCluster stopped.
